# Segmenting and Clustering Neighborhoods in Toronto

## To create the above dataframe:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have complete information and not greyed out or not assigned.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Page Layout of Postal Codes of Canada: M

The layout of this page appears to have changed since the original instructions were written. The page now presents the Postal Codes, the Boroughs and the Neighborhoods in simple table format:

![alt text][logo]
[logo]: ./postalcode_m.jpg "Postal Codes of Canada: M"


---

### Use Pandas `read_html` to get the table from the WikiPedia page
The table I am interested in is the first table on the page.

In [1]:
import pandas as pd

# Read the table
# The table headers are in row 0
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# Create the initial dataframe from the table
df = pd.DataFrame(data = table[0])

# Print the shape
print('The shape of the Raw Inital Datafram is: ', df.shape)

# Output the Head of the Table
df.head()

The shape of the Raw Inital Datafram is:  (289, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


---

## Handle rows where Borough is set but Neighbourhood is _Not assigned_

Some of the rows have Borough set but Neighbourhood is _Not assigned_. In this situation the Neighbourhood is to be set to the same value as the Borough.

In [2]:
# Find these instances
df[(df.Borough != 'Not assigned') & (df.Neighbourhood == 'Not assigned')]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


The is only one such value so it is easiest to manually fix

In [3]:
df.loc[df.Borough == "Queen's Park", 'Neighbourhood'] = "Queen's Park"

In [4]:
# Check again
df[(df.Borough != 'Not assigned') & (df.Neighbourhood == 'Not assigned')]

,Postcode,Borough,Neighbourhood


---

## Remove rows where Borough & Neighbourhood are _Not assigned_

In [5]:
df = df[(df.Borough != 'Not assigned') | (df.Neighbourhood != 'Not assigned')]

# Print the shape
print('The shape of the Raw Inital Datafram is: ', df.shape)

# Output the Head of the Table
df.head()

The shape of the Raw Inital Datafram is:  (212, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Count the distinct values in each colum

In [6]:
print('There are %d unique Postal Codes in the table' % df.Postcode.nunique())
print('There are %d unique Boroughs in the table' % df.Borough.nunique())
print('There are %d unique Neighbourhoods in the table' % df.Neighbourhood.nunique())

There are 103 unique Postal Codes in the table
There are 11 unique Boroughs in the table
There are 211 unique Neighbourhoods in the table


In [7]:
df.reset_index(drop=True, inplace=True)

---

## Some of the Neighbourhood values need to be cleaned up

There are issues with some of the Neighbourhood values containing the `]` character

In [8]:
df[df.Neighbourhood.str.contains(']')]

,Postcode,Borough,Neighbourhood
33,M1E,Scarborough,Guildwood]]
38,M6E,York,Caledonia-Fairbanks]]
114,M6N,York,]The Junction North]]
115,M6N,York,Runnymede]]


This is very easy to clean up

In [9]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace(']', '')

In [10]:
df[df.Neighbourhood.str.contains(']')]

,Postcode,Borough,Neighbourhood


----

## Group by Postal Code and Borough

The final task is to group by Postal Code and Borough and produce a list of all Neighbourhoods in each.

In [11]:
part_01 = pd.DataFrame(df.groupby(
    ['Postcode', 'Borough'])['Neighbourhood'].apply(
    lambda x: ', '.join(x))).reset_index()

In [12]:
part_01.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
part_01.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [14]:
part_01.shape

(103, 3)

---

# Get Latitude & Longitude for each Postal Code

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

The steps are:
1. Install geocoder if required
1. Create a list of Postal Codes
1. Search for the Latitude and Longitude of each postal code
1. Update the DataFrame with the Latitude and Longitude

In [ ]:
# Install geocoder if required
!conda install -c conda-forge geocoder --yes

In [79]:
# Only run once to get the Latitude / Longitude then Pickled
import geocoder

# Create a copy of the part_01 dataframe
part_02 = part_01.copy()

# Create a list of all the Postal Codes
postal_codes = part_01['Postcode'].tolist()

# Search for the Latitude and Longitude of each postal code
for postal_code in postal_codes:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    print('{}, Toronto, Ontario'.format(postal_code))
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      # g = geocoder.opencage('{}, Toronto, Ontario'.format(postal_code), key='13aa077d21ae42f287ec8607e07b2159')

      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    part_02.loc[part_02.Postcode == postal_code, 'Latitude'] = latitude
    part_02.loc[part_02.Postcode == postal_code, 'Longitude'] = longitude

M1B, Toronto, Ontario


ValueError: ('Subclass must define a valid URL. Got %s', 'https://maps.googleapis.com/maps/api/geocode/json')

In [ ]:
# Pickle the DataFrame to save having to make multiple calls
part_02.to_pickle('./toronto.pkl')

In [80]:
# Read from the Pickle File
part_02 = pd.read_pickle('./toronto.pkl')

In [16]:
# Show the new DataFrame for Part 02
part_02

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",44.385394,-79.691314
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.955403,-78.315606
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600
3,M1G,Scarborough,Woburn,43.644903,-79.381836
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900
5,M1J,Scarborough,Scarborough Village,43.746400,-79.232300
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.729800,-79.263900
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.712200,-79.284300
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724700,-79.231200
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.695200,-79.264600


---

# Segmenting and Clustering Neighborhoods in Toronto: Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. Add enough Markdown cells to explain what you decided to do and to report any observations you make.
1. Generate maps to visualize your neighborhoods and how they cluster together. 

### Import Required Libraries and setup FourSquare Veriables

In [63]:
### Import Folium for map rendering
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

# Library to handle JSON files
import json 

# Import Nominatim
# Converts an address into latitude and longitude values
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

# Library to handle requests
import requests 

# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Library to handle data in a vectorized manner
import numpy as np 

# Pandas library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Define Foursquare Credentials and Version
CLIENT_ID = 'EUDO0LACN3MV3DFIHGMVUMSWAILSHCBAOUVYP5EOGLXYTXTY'
CLIENT_SECRET = '41LFAJPS4WAQATSRKHGKOBPZRZBCUKB0TMWTR0VAJCIJJ3SD'
VERSION = '20180605'

# Limit of number of venues returned by Foursquare API
LIMIT = 100 
# Define radius of search for Foursquare API
radius = 1000 

### Helper Functions

In [18]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
# Function to get Nearby Venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

### Use the `getNearbyVenues` Function to get a table of venues

Then print the shape and the head ofthe dataframe to visualise the results

In [21]:
toronto_venues = getNearbyVenues(names=part_02['Postcode'],
                                 latitudes=part_02['Latitude'],
                                 longitudes=part_02['Longitude']
                                )

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(3460, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,44.385394,-79.691314,Lazy Tulip Cafe,44.388623,-79.690934,Café
1,M1B,44.385394,-79.691314,Casa Mia,44.388291,-79.693423,Italian Restaurant
2,M1B,44.385394,-79.691314,Donaleigh's Irish Public House,44.389344,-79.689112,Pub
3,M1B,44.385394,-79.691314,Spirit Catcher,44.386699,-79.689330,Sculpture Garden
4,M1B,44.385394,-79.691314,The Local Gastropub,44.388787,-79.691077,Gastropub


In [33]:
# Pickle the Venues DataFrame as well
toronto_venues.to_pickle('./toronto_venues.pkl')

In [46]:
toronto_venues = pd.read_pickle('./toronto_venues.pkl')

Let's check how many venues were returned for each Postal Code

In [47]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,28,28,28,28,28,28
M1C,2,2,2,2,2,2
M1E,27,27,27,27,27,27
M1G,100,100,100,100,100,100
M1H,2,2,2,2,2,2
M1J,2,2,2,2,2,2
M1K,16,16,16,16,16,16
M1L,9,9,9,9,9,9
M1M,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


There are a lot of Postal codes with fewer than 10 venues. Let's filter them out.

In [67]:
size = toronto_venues.Neighborhood.value_counts()
idx = size[size >= 10].index
toronto_venues_filtered = toronto_venues[toronto_venues.Neighborhood.isin(idx)]
toronto_venues_filtered.shape

(3262, 7)

## 3. Analyze Each Postal Code

In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_filtered[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move Neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns= cols)

In [57]:
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [58]:
toronto_onehot.shape

(3262, 241)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [60]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00000,0.107143,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(54, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Coffee Shop,Pub,Pizza Place,Hotel,Smoke Shop,Sandwich Place,Café,Restaurant,Poutine Place,Mexican Restaurant
1,M1E,Coffee Shop,Fast Food Restaurant,Smoothie Shop,Breakfast Spot,Electronics Store,Supermarket,Fried Chicken Joint,Sports Bar,Restaurant,Medical Center
2,M1G,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
3,M1K,Coffee Shop,Discount Store,Bank,Chinese Restaurant,Sandwich Place,Light Rail Station,Train Station,Intersection,Department Store,Hobby Shop
4,M1M,Ice Cream Shop,Bank,Pizza Place,Sandwich Place,Bistro,Coffee Shop,Discount Store,Flower Shop,Pharmacy,Department Store
5,M1T,Pharmacy,Pizza Place,Thai Restaurant,Rental Car Location,Fast Food Restaurant,Bus Stop,Fried Chicken Joint,Chinese Restaurant,Coffee Shop,Italian Restaurant
6,M1W,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
7,M2J,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
8,M2M,Japanese Restaurant,Café,Korean Restaurant,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fast Food Restaurant,Ramen Restaurant
9,M2N,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall


Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [64]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 0, 2, 3, 0, 0, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [78]:
toronto_merged = part_02[part_02.Postcode.isin(idx)].copy()
toronto_merged.rename(columns={'Postcode': 'Neighborhood'}, inplace=True)

# Add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged

(54, 16)


,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",44.385394,-79.691314,2,Coffee Shop,Pub,Pizza Place,Hotel,Smoke Shop,Sandwich Place,Café,Restaurant,Poutine Place,Mexican Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600,2,Coffee Shop,Fast Food Restaurant,Smoothie Shop,Breakfast Spot,Electronics Store,Supermarket,Fried Chicken Joint,Sports Bar,Restaurant,Medical Center
3,M1G,Scarborough,Woburn,43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.729800,-79.263900,0,Coffee Shop,Discount Store,Bank,Chinese Restaurant,Sandwich Place,Light Rail Station,Train Station,Intersection,Department Store,Hobby Shop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724700,-79.231200,2,Ice Cream Shop,Bank,Pizza Place,Sandwich Place,Bistro,Coffee Shop,Discount Store,Flower Shop,Pharmacy,Department Store
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781200,-79.303600,3,Pharmacy,Pizza Place,Thai Restaurant,Rental Car Location,Fast Food Restaurant,Bus Stop,Fried Chicken Joint,Chinese Restaurant,Coffee Shop,Italian Restaurant
15,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall
21,M2M,North York,"Newtonbrook, Willowdale",43.763531,-79.411147,2,Japanese Restaurant,Café,Korean Restaurant,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Fast Food Restaurant,Ramen Restaurant
22,M2N,North York,Willowdale South,43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,Sports Bar,Gym / Fitness Center,Concert Hall


Finally, let's visualize the resulting clusters

In [76]:
# create map
toronto_latitude = 43.70011
toronto_longitude = -79.4163
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters